In [2]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Path to the CSV file in your Google Drive
drive_csv_path = '/content/drive/My Drive/fer2013.csv'

# Read CSV file into DataFrame
df = pd.read_csv(drive_csv_path)

# Display the first few rows of the DataFrame
print(df.head())


Mounted at /content/drive
   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [3]:
import os
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

In [4]:
df.emotion.unique()

array([0, 2, 4, 6, 3, 5, 1])

In [20]:
labels_to_text = {
    0: 'anger',
    1: 'disgust',
    2: 'fear',
    3: 'happiness',
    4: 'sadness',
    5: 'surprise',
    6: 'neutral'
}


In [6]:
import numpy as np
np.array(df.pixels.loc[0].split(' ')).reshape(48,48)

array([['70', '80', '82', ..., '52', '43', '41'],
       ['65', '61', '58', ..., '56', '52', '44'],
       ['50', '43', '54', ..., '49', '56', '47'],
       ...,
       ['91', '65', '42', ..., '72', '56', '43'],
       ['77', '82', '79', ..., '105', '70', '46'],
       ['77', '72', '84', ..., '106', '109', '82']], dtype='<U3')

In [7]:
img_array= df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48,48,1).astype('float32'))

In [8]:
img_array= np.stack(img_array,axis=0)

In [9]:
labels= df.emotion.values

In [29]:
X_train,X_test, y_train, y_test = train_test_split(img_array,labels,test_size=.2)

In [30]:
X_test.shape

(7178, 48, 48, 1)

In [24]:
y_test.shape

(7178,)

In [13]:
X_train= X_train/255
X_test= X_test/255

In [14]:
basemodel = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),


    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(7,activation='softmax')


])


In [15]:
basemodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 23, 23, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                        

In [16]:
basemodel.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


In [17]:
import os

try:
    os.mkdir('checkpoint')
except:
    pass


In [18]:

file_name = 'best_model.h5'
checkpoint_path = os.path.join('checkpoint', file_name)

call_back = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                               monitor='val_accuracy',
                                               verbose=1,
                                               save_freq='epoch',
                                               save_best_only=True,
                                               save_weights_only=False,
                                               mode='max')

In [ ]:
basemodel.fit(X_train,y_train,epochs=20,validation_split=.1,callbacks=call_back)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Input, BatchNormalization, Dropout, Dense
from tensorflow.keras.models import Model

# Define your provided model architecture
def create_model(input_shape=(48,48,1), num_classes=7):
    input = Input(shape=input_shape)
    x = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(input)
    x = Conv2D(filters=512, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=384, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=192, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=384, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.4)(x)

    x = Flatten()(x)

    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(num_classes, activation='softmax')(x)

    return Model(input, x, name='fer_model')

# Load your data and preprocess it
# Assuming you have already defined X_train, y_train, X_test, y_test

# Normalize pixel values
X_train = X_train / 255
X_test = X_test / 255

# Convert integer labels to one-hot encoded vectors
y_train_encoded = to_categorical(y_train, num_classes=7)
y_test_encoded = to_categorical(y_test, num_classes=7)

# Create the model
model = create_model()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Set up model checkpoint callback
file_name = 'best_model.h5'
checkpoint_path = os.path.join('checkpoint', file_name)
call_back = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                               monitor='val_accuracy',
                                               verbose=1,
                                               save_freq='epoch',
                                               save_best_only=True,
                                               save_weights_only=False,
                                               mode='max')

# Train the model
history = model.fit(X_train, y_train_encoded, epochs=20, validation_split=0.2, callbacks=[call_back])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


Epoch 1/20
718/718 [==============================] - ETA: 0s - loss: 1.9657 - accuracy: 0.2719
Epoch 1: val_accuracy improved from -inf to 0.26228, saving model to checkpoint/best_model.h5
718/718 [==============================] - 92s 114ms/step - loss: 1.9657 - accuracy: 0.2719 - val_loss: 1.9253 - val_accuracy: 0.2623
Epoch 2/20
718/718 [==============================] - ETA: 0s - loss: 1.5161 - accuracy: 0.4159
Epoch 2: val_accuracy improved from 0.26228 to 0.37252, saving model to checkpoint/best_model.h5
718/718 [==============================] - 77s 107ms/step - loss: 1.5161 - accuracy: 0.4159 - val_loss: 1.6472 - val_accuracy: 0.3725
Epoch 3/20
718/718 [==============================] - ETA: 0s - loss: 1.3164 - accuracy: 0.5004
Epoch 3: val_accuracy improved from 0.37252 to 0.50644, saving model to checkpoint/best_model.h5
718/718 [==============================] - 78s 109ms/step - loss: 1.3164 - accuracy: 0.5004 - val_loss: 1.3465 - val_accuracy: 0.5064
Epoch 4/20
718/718 [==